# Library

In [22]:
# Installation
%pip install --quiet mesa

Note: you may need to restart the kernel to use updated packages.


In [23]:
# Import
import mesa
import numpy as np
import pandas as pd

import math
import random

# Class Definition

## Agent

In [24]:
class ChartistAgent(mesa.Agent):
    def __init__(self, id, model, fiat_owned, bitcoin_owned, chartist_day_reference):
        super().__init__(id, model)

        # Attribute Initialization         
        self.fiat = fiat_owned
        self.bitcoin = bitcoin_owned
        self.is_close = True
        self.n = chartist_day_reference
        
        # Default Rule : EMA for both opening and closing
        self.is_open_sma_high = False
        self.is_close_sma_high = False
        
        if (np.random.uniform() <= 0.25):
            # Rule : SMA for both opening and closing             
            self.is_open_sma_high = True
            self.is_close_sma_high = True
        elif (np.random.uniform() > 0.25 and np.random.uniform() < 0.5):
            # Rule : SMA for opening and EMA for closing
            self.is_open_sma_high = True
        elif (np.random.uniform() > 0.25 and np.random.uniform() < 0.75):
            # Rule : EMA for opening and SMA for closing
            self.is_close_sma_high = True
    
    def step(self):
        if self.is_close:
            # Potential Buy
            sma_value = self.calculate_sma_n_days()
            ema_value = self.calculate_ema_n_days()
            
            # Count Probability to Buy Based on Rule
            probability_to_buy = 0
            if self.model.price < sma_value:
                probability_to_buy += (0.8 if self.is_open_sma_high else 0.2)
            
            if self.model.price > ema_value:
                probability_to_buy += (0.8 if not self.is_open_sma_high else 0.2)
                
            print(f'{self.unique_id} : {self.is_open_sma_high} {self.is_close_sma_high}')
            print(f'sma value : {sma_value}, ema value : {ema_value}')
            print(f'price : {self.model.price}')
            print(f'prob : {probability_to_buy}')
            
            # Actual Buy
            if (np.random.uniform() <= probability_to_buy):
                self.is_close = False
                
                print('PREVIOUS STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')
                
                bitcoin_obtained = math.floor((self.fiat / self.model.price) * 10000) / 10000
                self.bitcoin += bitcoin_obtained
                self.fiat -= self.bitcoin * self.model.price
                self.model.demand += bitcoin_obtained

                print('AFTER STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')
    
        else:
            # Potential Sell
            sma_value = self.calculate_sma_n_days()
            ema_value = self.calculate_ema_n_days()

            # Count Probability to Sell Based on Rule
            probability_to_sell = 0
            if self.model.price > sma_value:
                probability_to_sell += (0.8 if self.is_close_sma_high else 0.2)
            
            if self.model.price < ema_value:
                probability_to_sell += (0.8 if not self.is_close_sma_high else 0.2)
        
            print(f'{self.unique_id} : {self.is_open_sma_high} {self.is_close_sma_high}')
            print(f'sma value : {sma_value}, ema value : {ema_value}')
            print(f'price : {self.model.price}')
            print(f'prob : {probability_to_sell}')
            
            # Actual Sell
            if (np.random.uniform() <= probability_to_sell):
                self.is_close = True
                
                print('PREVIOUS STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')
                
                bitcoin_released = self.bitcoin
                self.fiat += self.bitcoin * self.model.price
                self.bitcoin -= bitcoin_released
                self.model.supply += bitcoin_released
                
                print('AFTER STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')
                    
    # Agent Helper Function
    # SMA Rule
    def calculate_sma_n_days(self):
        start_index = max(0, len(self.model.price_history) - self.n)
        window = self.model.price_history[start_index:len(self.model.price_history)]

        return sum(window) / len(window)

    # EMA Rule
    def calculate_ema_n_days(self):
        smooth_factor = 2 / (self.n + 1)
        ema = [self.model.price_history[0]]

        for i in range(1, len(self.model.price_history)):
            value = smooth_factor * self.model.price_history[i] + (1 - smooth_factor) * ema[i-1]
            ema.append(value)

        return ema[len(ema)-1]

In [25]:
class RandomAgent(mesa.Agent):
    def __init__(self, id, model, fiat_owned, bitcoin_owned):
        super().__init__(id, model)

        # Attribute Initialization         
        self.fiat = fiat_owned
        self.bitcoin = bitcoin_owned
        self.is_close = True
    
    def step(self):
        print(f'{self.unique_id}')

        if self.is_close:
            # Actual Buy
            if (random.choice([True, False])):
                self.is_close = False

                print(f'BUY')
                print('PREVIOUS STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')

                bitcoin_obtained = math.floor((self.fiat / self.model.price) * 10000) / 10000
                self.bitcoin += bitcoin_obtained
                self.fiat -= self.bitcoin * self.model.price
                self.model.demand += bitcoin_obtained
                
                print('AFTER STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')
    
        else:
            # Actual Sell
            if (random.choice([True, False])):
                self.is_close = True
                
                print(f'SELL')
                print('PREVIOUS STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')
                
                bitcoin_released = self.bitcoin
                self.fiat += self.bitcoin * self.model.price
                self.bitcoin -= bitcoin_released
                self.model.supply += bitcoin_released
                
                print('AFTER STATE')
                print(f'bitcoin : {self.bitcoin}')
                print(f'fiat : {self.fiat}')


In [26]:
import random

class BitcoinMarketModel(mesa.Model):
    def __init__(self, price_start, number_of_agents, total_fiat, total_bitcoin, chartist_ratio, chartist_day_reference):
        self.schedule = mesa.time.RandomActivation(self)
        
        self.price = price_start
        self.price_history = [price_start]
        self.number_of_agents = number_of_agents
        
        self.total_fiat = total_fiat
        self.total_bitcoin = total_bitcoin
        self.supply = 0
        self.demand = 0

        for i in range(self.number_of_agents):
            fiat_owned = self.total_fiat / self.number_of_agents
            bitcoin_owned = 0
            
            if (np.random.uniform() <= chartist_ratio):
                a = ChartistAgent(str(f"chartist-{i}"), 
                              self, 
                              fiat_owned,
                              bitcoin_owned, 
                              chartist_day_reference)

                self.schedule.add(a)
            else:
                a = RandomAgent(str(f"random-{i}"), 
                              self, 
                              fiat_owned,
                              bitcoin_owned)

                self.schedule.add(a)
    
    def step(self):
        # Before Stepping
        print('NEW CYCLE')
        self.price += self.calculate_today_price_change()
        self.supply = 0
        self.demand = 0
        
        # Stepping
        self.schedule.step()
        
        # After Stepping
        self.price_history.append(self.price)

        
    # Model Helper Function
    # Calculate Price
    def calculate_today_price_change(self):
        supply_demand_difference = self.demand - self.supply
        sign = math.copysign(1, supply_demand_difference)
        
        return math.floor((math.sqrt(2) / 2) *  sign * math.sqrt(abs(supply_demand_difference)))
    
    # Calculate New Bitcoin Introduced to Model
    def calculate_new_bitcoin(self):
        return 0.6 * self.total_bitcoin

# SIMULATION

In [27]:
# total_bitcoin = 10000
# total_fiat = 5000000

# # Agents Parameters
# number_of_agents = 5 # In 2020, there are 420M users
# chartist_day_reference = 3
# chartist_ratio = 0.6

# price_start = 5614

# model = BitcoinMarketModel(price_start, 
#                            number_of_agents, 
#                            total_fiat, 
#                            total_bitcoin, 
#                            chartist_ratio, 
#                            chartist_day_reference)

# for i in range(10):
#     print(f'progress : {i}/1399')
#     model.step()
    
# agents = model.schedule.agents
# sum_fiat = 0
# sum_bitcoin = 0
# count_open = 0

# sum_chartist_fiat = 0
# sum_chartist_bitcoin = 0
# count_chartist_open = 0
# count_chartist_agent = 0

# sum_chartist_R1R1_fiat = 0
# sum_chartist_R1R1_bitcoin = 0
# count_chartist_R1R1_open = 0
# count_chartist_R1R1_agent = 0

# sum_chartist_R2R1_fiat = 0
# sum_chartist_R2R1_bitcoin = 0
# count_chartist_R2R1_open = 0
# count_chartist_R2R1_agent = 0

# sum_chartist_R1R2_fiat = 0
# sum_chartist_R1R2_bitcoin = 0
# count_chartist_R1R2_open = 0
# count_chartist_R1R2_agent = 0

# sum_chartist_R2R2_fiat = 0
# sum_chartist_R2R2_bitcoin = 0
# count_chartist_R2R2_open = 0
# count_chartist_R2R2_agent = 0

# sum_random_fiat = 0
# sum_random_bitcoin = 0
# count_random_open = 0

# for a in agents:
#     sum_fiat += a.fiat
#     sum_bitcoin += a.bitcoin
#     if a.bitcoin > 0:
#         count_open += 1
        
#     if 'chartist' in a.unique_id:
#         sum_chartist_fiat += a.fiat
#         sum_chartist_bitcoin += a.bitcoin
#         count_chartist_agent += 1
        
#         if a.bitcoin > 0:
#             count_chartist_open += 1
        
#         if a.is_open_filtering_high and a.is_close_filtering_high:
#             sum_chartist_R1R1_fiat += a.fiat
#             sum_chartist_R1R1_bitcoin += a.bitcoin
#             count_chartist_R1R1_agent += 1

#             if a.bitcoin > 0:
#                 count_chartist_R1R1_open += 1
        
#         if a.is_open_filtering_high and not a.is_close_filtering_high:
#             sum_chartist_R1R2_fiat += a.fiat
#             sum_chartist_R1R2_bitcoin += a.bitcoin
#             count_chartist_R1R2_agent += 1

#             if a.bitcoin > 0:
#                 count_chartist_R1R2_open += 1
                
#         if not a.is_open_filtering_high and a.is_close_filtering_high:
#             sum_chartist_R2R1_fiat += a.fiat
#             sum_chartist_R2R1_bitcoin += a.bitcoin
#             count_chartist_R2R1_agent += 1

#             if a.bitcoin > 0:
#                 count_chartist_R2R1_open += 1
        
#         if not a.is_open_filtering_high and not a.is_close_filtering_high:
#             sum_chartist_R2R2_fiat += a.fiat
#             sum_chartist_R2R2_bitcoin += a.bitcoin
#             count_chartist_R2R2_agent += 1

#             if a.bitcoin > 0:
#                 count_chartist_R2R2_open += 1
            
    
#     if 'random' in a.unique_id:
#         sum_random_fiat += a.fiat
#         sum_random_bitcoin += a.bitcoin
#         if a.bitcoin > 0:
#             count_random_open += 1

# price = model.price
# bitcoin_value = sum_bitcoin * price
# total_valuation = sum_fiat + bitcoin_value
# print(f"Sum Fiat {sum_fiat}")
# print(f"Sum Bitcoin {sum_bitcoin}")
# print(f"Value Bitcoin {bitcoin_value}")
# print(f"Total Valuation {total_valuation}")
# print(f"Count Open {count_open}")

# chartist_bitcoin_value = sum_chartist_bitcoin * price
# total_valuation_chartist = sum_chartist_fiat + chartist_bitcoin_value
# print(f"Sum Chartist Fiat {sum_chartist_fiat}")
# print(f"Sum Chartist Bitcoin {sum_chartist_bitcoin}")
# print(f"Value Chartist Bitcoin {chartist_bitcoin_value}")
# print(f"Total Chartist Valuation {total_valuation_chartist}")
# print(f"Count Chartist Open {count_chartist_open}")

# chartist_R1R1_bitcoin_value = sum_chartist_R1R1_bitcoin * price
# total_valuation_chartist_R1R1 = sum_chartist_R1R1_fiat + chartist_R1R1_bitcoin_value
# print(f"Sum Chartist R1R1 Fiat {sum_chartist_R1R1_fiat}")
# print(f"Sum Chartist R1R1 Bitcoin {sum_chartist_R1R1_bitcoin}")
# print(f"Value Chartist R1R1 Bitcoin {chartist_R1R1_bitcoin_value}")
# print(f"Total Chartist R1R1 Valuation {total_valuation_chartist_R1R1}")
# print(f"Count Chartist R1R1 Open {count_chartist_R1R1_open}")

# chartist_R1R2_bitcoin_value = sum_chartist_R1R2_bitcoin * price
# total_valuation_chartist_R1R2 = sum_chartist_R1R2_fiat + chartist_R1R2_bitcoin_value
# print(f"Sum Chartist R1R2 Fiat {sum_chartist_R1R2_fiat}")
# print(f"Sum Chartist R1R2 Bitcoin {sum_chartist_R1R2_bitcoin}")
# print(f"Value Chartist R1R2 Bitcoin {chartist_R1R2_bitcoin_value}")
# print(f"Total Chartist R1R2 Valuation {total_valuation_chartist_R1R2}")
# print(f"Count Chartist R1R2 Open {count_chartist_R1R2_open}")

# chartist_R2R1_bitcoin_value = sum_chartist_R2R1_bitcoin * price
# total_valuation_chartist_R2R1 = sum_chartist_R2R1_fiat + chartist_R2R1_bitcoin_value
# print(f"Sum Chartist R2R1 Fiat {sum_chartist_R2R1_fiat}")
# print(f"Sum Chartist R2R1 Bitcoin {sum_chartist_R2R1_bitcoin}")
# print(f"Value Chartist R2R1 Bitcoin {chartist_R2R1_bitcoin_value}")
# print(f"Total Chartist R2R1 Valuation {total_valuation_chartist_R2R1}")
# print(f"Count Chartist R2R1 Open {count_chartist_R2R1_open}")

# chartist_R2R2_bitcoin_value = sum_chartist_R2R2_bitcoin * price
# total_valuation_chartist_R2R2 = sum_chartist_R2R2_fiat + chartist_R2R2_bitcoin_value
# print(f"Sum Chartist R2R2 Fiat {sum_chartist_R2R2_fiat}")
# print(f"Sum Chartist R2R2 Bitcoin {sum_chartist_R2R2_bitcoin}")
# print(f"Value Chartist R2R2 Bitcoin {chartist_R2R2_bitcoin_value}")
# print(f"Total Chartist R2R2 Valuation {total_valuation_chartist_R2R2}")
# print(f"Count Chartist R2R2 Open {count_chartist_R2R2_open}")

# random_bitcoin_value = sum_random_bitcoin * price
# total_valuation_random = sum_random_fiat + random_bitcoin_value
# print(f"Sum Random Fiat {sum_random_fiat}")
# print(f"Sum Random Bitcoin {sum_random_bitcoin}")
# print(f"Value Random Bitcoin {random_bitcoin_value}")
# print(f"Total Random Valuation {total_valuation_random}")
# print(f"Count Random Open {count_random_open}")

# print(f"NUMBER OF CHARTIST : {count_chartist_agent}")
# print(f"Chartist vs All Proportion Fiat : {sum_chartist_fiat / sum_fiat  * 100}")
# print(f"Chartist vs All Proportion Bitcoin : {sum_chartist_bitcoin / sum_bitcoin  * 100}")
# print(f"Chartist vs All Proportion Total Valuation : {total_valuation_chartist / total_valuation  * 100}")

# print(f"NUMBER OF CHARTIST R1R1 : {count_chartist_R1R1_agent}")
# print(f"Chartist R1R1 vs Chartist All Proportion Fiat : {sum_chartist_R1R1_fiat / sum_chartist_fiat  * 100}")
# print(f"Chartist R1R1 vs Chartist All Proportion Bitcoin : {sum_chartist_R1R1_bitcoin / sum_chartist_bitcoin  * 100}")
# print(f"Chartist R1R1 vs Chartist All Proportion Total Valuation : {total_valuation_chartist_R1R1 / total_valuation_chartist  * 100}")

# print(f"NUMBER OF CHARTIST R1R1 : {count_chartist_R1R2_agent}")
# print(f"Chartist R1R2 vs Chartist All Proportion Fiat : {sum_chartist_R1R2_fiat / sum_chartist_fiat  * 100}")
# print(f"Chartist R1R2 vs Chartist All Proportion Bitcoin : {sum_chartist_R1R2_bitcoin / sum_chartist_bitcoin  * 100}")
# print(f"Chartist R1R2 vs Chartist All Proportion Total Valuation : {total_valuation_chartist_R1R2 / total_valuation_chartist  * 100}")

# print(f"NUMBER OF CHARTIST R1R1 : {count_chartist_R2R1_agent}")
# print(f"Chartist R2R1 vs Chartist All Proportion Fiat : {sum_chartist_R2R1_fiat / sum_chartist_fiat  * 100}")
# print(f"Chartist R2R1 vs Chartist All Proportion Bitcoin : {sum_chartist_R2R1_bitcoin / sum_chartist_bitcoin  * 100}")
# print(f"Chartist R2R1 vs Chartist All Proportion Total Valuation : {total_valuation_chartist_R2R1 / total_valuation_chartist  * 100}")

# print(f"NUMBER OF CHARTIST R1R1 : {count_chartist_R2R2_agent}")
# print(f"Chartist R2R2 vs Chartist All Proportion Fiat : {sum_chartist_R2R2_fiat / sum_chartist_fiat  * 100}")
# print(f"Chartist R2R2 vs Chartist All Proportion Bitcoin : {sum_chartist_R2R2_bitcoin / sum_chartist_bitcoin  * 100}")
# print(f"Chartist R2R2 vs Chartist All Proportion Total Valuation : {total_valuation_chartist_R2R2 / total_valuation_chartist  * 100}")


In [28]:
total_bitcoin = 10000
total_fiat = 5000000

# Agents Parameters
number_of_agents = 5 # In 2020, there are 420M users
chartist_day_reference = 3
chartist_ratio = 0.5

price_start = 5614

model = BitcoinMarketModel(price_start, 
                           number_of_agents, 
                           total_fiat, 
                           total_bitcoin, 
                           chartist_ratio, 
                           chartist_day_reference)

for i in range(10):
    print(f'progress : {i}/1399')
    model.step()

progress : 0/1399
NEW CYCLE
chartist-1 : True False
sma value : 5614.0, ema value : 5614
price : 5614
prob : 0
chartist-3 : True False
sma value : 5614.0, ema value : 5614
price : 5614
prob : 0
random-4
chartist-2 : True False
sma value : 5614.0, ema value : 5614
price : 5614
prob : 0
chartist-0 : True True
sma value : 5614.0, ema value : 5614
price : 5614
prob : 0
progress : 1/1399
NEW CYCLE
random-4
chartist-2 : True False
sma value : 5614.0, ema value : 5614.0
price : 5614
prob : 0
chartist-3 : True False
sma value : 5614.0, ema value : 5614.0
price : 5614
prob : 0
chartist-1 : True False
sma value : 5614.0, ema value : 5614.0
price : 5614
prob : 0
chartist-0 : True True
sma value : 5614.0, ema value : 5614.0
price : 5614
prob : 0
progress : 2/1399
NEW CYCLE
chartist-2 : True False
sma value : 5614.0, ema value : 5614.0
price : 5614
prob : 0
chartist-1 : True False
sma value : 5614.0, ema value : 5614.0
price : 5614
prob : 0
random-4
chartist-0 : True True
sma value : 5614.0, ema va